In [ ]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке, 
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
# 
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию, 
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными, 
# которые необходимо произвести, что бы сопоставить признаки целевому значению.

### Область работы 1 (библиотеки)

In [ ]:
# Данный блок в области 1 выполняется преподавателем
# 
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
#
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
#
# pip install

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#pip install category_encoders
import category_encoders as ce

from sklearn.model_selection import train_test_split , GridSearchCV , RandomizedSearchCV, KFold, RepeatedKFold

from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder , LabelEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import get_scorer
from sklearn.model_selection import cross_validate
from sklearn.model_selection import learning_curve
from matplotlib.colors import ListedColormap
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
from sklearn import svm 
from sklearn.svm import LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from itertools import product
from sklearn.ensemble import VotingClassifier
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.inspection import permutation_importance
from IPython.display import display
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report, RocCurveDisplay, DetCurveDisplay,PrecisionRecallDisplay
from sklearn.metrics import roc_auc_score

# 2 БЛОК ВЫПОЛНЕН В ДРУГОМ ФАЙЛЕ

### Область работы 3 (выполнение лучшей модели)

In [ ]:
# Данный блок(и) в области 3 выполняется преподавателем
#
# В области находится одна, единственная, итоговая модель машинного обучения с однозначными, 
# зафиксированными параметрами
#
# В данной области категорически запрещается искать, выбирать, улучшать, оптимизировать, 
# тюниговать и т.д. модель машинного обучения

In [ ]:
# Путь к тренировочному набору
path_train = pd.read_csv('income_train.csv')
df = pd.DataFrame(path_train)
df['income'] = np.where(df['income'] == '>50K',1,0)
display(df.head(),df.shape)

path_test = pd.read_csv('income_test.csv')
df1 = pd.DataFrame(path_test)
display(df1.head())
df1.shape

## Делаем обработку df (т.е train)

In [ ]:
df = df.replace('?',np.NaN)
df = df.dropna()
# это выброс, тк рядом значений нет, а самое близкое к этому это где-то 43000
df = df.drop(df[df['capital-gain'] == 99999].index)
display(df.head(),df.shape)

In [ ]:
#### Relationship
df.relationship = df.relationship.replace('Husband','Family')
df.relationship = df.relationship.replace('Wife','Family')
df.relationship = df.relationship.replace('Unmarried','Not-in-family')
df.relationship = df.relationship.replace('Own-child','Other')
df.relationship = df.relationship.replace('Other-relative','Other')
###### Occupation
df.occupation = df.occupation.replace('Prof-specialty','Professional')
df.occupation = df.occupation.replace('Craft-repair','Physical labor') # blue - рабочий класс(почасовая полата) - физич труд
df.occupation = df.occupation.replace('Exec-managerial','Intellectual labor') # white - наемный работник(доход в месяц) - умственный
df.occupation = df.occupation.replace('Adm-clerical','Intellectual labor') # white
df.occupation = df.occupation.replace('Machine-op-inspct','Physical labor') # blue
df.occupation = df.occupation.replace('Transport-moving','Physical labor')# blue
df.occupation = df.occupation.replace('Handlers-cleaners','Physical labor') # blue
df.occupation = df.occupation.replace('Farming-fishing','Physical labor')# blue
df.occupation = df.occupation.replace('Tech-support','Other-service')
df.occupation = df.occupation.replace('Protective-serv','Other-service')
df.occupation = df.occupation.replace('Priv-house-serv','Other-service')
df.occupation = df.occupation.replace('Armed-Forces','Other-service')
##### Workclass
df.workclass = df.workclass.replace('Federal-gov','Government')
df.workclass = df.workclass.replace('State-gov','Government')
df.workclass = df.workclass.replace('Local-gov','Government')
df.workclass = df.workclass.replace('Self-emp-inc','Self-Employed')
df.workclass = df.workclass.replace('Self-emp-not-inc','Self-Employed')
df.workclass = df.workclass.replace('Never-worked', 'Other')
df.workclass = df.workclass.replace('Without-pay', 'Other')
#### Marital
df['marital-status'] = df['marital-status'].replace('Married-civ-spouse','Married')
df['marital-status'] = df['marital-status'].replace('Married-AF-spouse','Married')
df['marital-status'] = df['marital-status'].replace('Married-spouse-absent','Married')
df['marital-status'] = df['marital-status'].replace('Never-married','Single')
##### Race
df.race = df.race.replace('Black','Other')
df.race = df.race.replace('Asian-Pac-Islander','Other')
df.race = df.race.replace('Amer-Indian-Eskimo','Other')
##### Country
country = df['native-country'].unique()
def zamena_country(data):
    for i in country:
        if i != 'United-States':
            data['native-country'] = data['native-country'].replace(i,'Other')
    return data
df = zamena_country(df)

###### Capital changes
df['capital-changes']  = df['capital-gain'] - df['capital-loss']
##### 
df = df.drop(columns=['education','fnlwgt','capital-gain','capital-loss'])
df.head()

## Column Transformer

In [ ]:
num_features = ['age','hours-per-week','capital-changes']
y = np.array(df.income)
X = df.drop(columns=['income'])

In [ ]:
workclass_map = [{
    'col':'workclass', 
    'mapping':{ 'Other':3, 'Government':2, 'Self-Employed':1, 'Private':0}
    }]
occupation_map = [{
    'col':'occupation',
    'mapping':{'Other-service':4, 'Sales':3,'Physical labor':2, 'Professional':1, 'Intellectual labor':0}
    }]
marital_map = [{
    'col':'marital-status',
    'mapping':{'Separated': 4, 'Widowed': 3, 'Divorced':2, 'Single': 1, 'Married':0}
    }]
sex_map = [{
    'col':'sex',
    'mapping':{'Female':1, 'Male':0}
    }]
race_map = [{
    'col':'race',
    'mapping':{'Other':1, 'White':0}
    }]
native_country_map = [{
    'col':'native-country',
    'mapping':{'Other':1, 'United-States':0}
    }]
education_map = [{
    'col':'education',
    'mapping':{'Academia':2, 'School':1, 'Higher-educ':0}
    }]
relationship_map = [{
    'col':'relationship',
    'mapping':{'Other':2, 'Not-in-family':1, 'Family':0}
    }]

sex_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = sex_map))])

workclass_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = workclass_map))])

marital_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = marital_map))])

occupation_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = occupation_map))])

race_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = race_map))])

native_country_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = native_country_map))])
education_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = education_map))])
relationship_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = relationship_map))])

   
# для деревьев можно не стандартизировать данные в фичах 
CT = ColumnTransformer(transformers=[('sex', sex_transformer, ['sex']),
                                     ('work', workclass_transformer, ['workclass']),
                                     ('matiral', marital_transformer, ['marital-status']),
                                     ('occupation', occupation_transformer, ['occupation']),
                                     ('race', race_transformer, ['race']),
                                     ('native-country',native_country_transformer, ['native-country']),
                                     ('relationship',relationship_transformer,['relationship'])],
                                      remainder = 'passthrough')

display(CT)

## Делаем обработку df1 (т.е test)

In [ ]:
# более наглядная обработка и нахождение выбросов и nan в файле 2 блока
# тут сразу буду чистить данные

In [ ]:
# не выбрасываем выбросы и ? значения, а добавляем их в другие категории к существующим
display(df1.head(),df1.shape)

In [ ]:
#### Relationship
df1.relationship = df1.relationship.replace('Husband','Family')
df1.relationship = df1.relationship.replace('Wife','Family')
df1.relationship = df1.relationship.replace('Unmarried','Not-in-family')
df1.relationship = df1.relationship.replace('Own-child','Other')
df1.relationship = df1.relationship.replace('Other-relative','Other')
###### Occupation
df1.occupation = df1.occupation.replace('Prof-specialty','Professional')
df1.occupation = df1.occupation.replace('Craft-repair','Physical labor') # blue - рабочий класс(почасовая полата) - физич труд
df1.occupation = df1.occupation.replace('Exec-managerial','Intellectual labor') # white - наемный работник(доход в месяц) - умственный
df1.occupation = df1.occupation.replace('Adm-clerical','Intellectual labor') # white
df1.occupation = df1.occupation.replace('Machine-op-inspct','Physical labor') # blue
df1.occupation = df1.occupation.replace('Transport-moving','Physical labor')# blue
df1.occupation = df1.occupation.replace('Handlers-cleaners','Physical labor') # blue
df1.occupation = df1.occupation.replace('Farming-fishing','Physical labor')# blue
df1.occupation = df1.occupation.replace('Tech-support','Other-service')
df1.occupation = df1.occupation.replace('Protective-serv','Other-service')
df1.occupation = df1.occupation.replace('Priv-house-serv','Other-service')
df1.occupation = df1.occupation.replace('Armed-Forces','Other-service')
df1.occupation = df1.occupation.replace('?','Other-service')
##### Workclass
df1.workclass = df1.workclass.replace('Federal-gov','Government')
df1.workclass = df1.workclass.replace('State-gov','Government')
df1.workclass = df1.workclass.replace('Local-gov','Government')
df1.workclass = df1.workclass.replace('Self-emp-inc','Self-Employed')
df1.workclass = df1.workclass.replace('Self-emp-not-inc','Self-Employed')
df1.workclass = df1.workclass.replace('Never-worked', 'Other')
df1.workclass = df1.workclass.replace('Without-pay', 'Other')
df1.workclass = df1.workclass.replace('?', 'Other')
#### Marital
df1['marital-status'] = df1['marital-status'].replace('Married-civ-spouse','Married')
df1['marital-status'] = df1['marital-status'].replace('Married-AF-spouse','Married')
df1['marital-status'] = df1['marital-status'].replace('Married-spouse-absent','Married')
df1['marital-status'] = df1['marital-status'].replace('Never-married','Single')
##### Race
df1.race = df1.race.replace('Black','Other')
df1.race = df1.race.replace('Asian-Pac-Islander','Other')
df1.race = df1.race.replace('Amer-Indian-Eskimo','Other')
##### Country
countries = df1['native-country'].unique()
def zamena_countries(data):
    for i in countries:
        if i != 'United-States':
            data['native-country'] = data['native-country'].replace(i,'Other')
    return data
df1 = zamena_countries(df1)

###### Capital changes
df1['capital-changes']  = df1['capital-gain'] - df1['capital-loss']
##### 
df1 = df1.drop(columns=['education','fnlwgt','capital-gain','capital-loss'])
display(df1.head(),df1.shape)

In [ ]:
# Блок(и) обучения и поверки модели

In [ ]:
finish_pipe_RFR =  Pipeline([
            ('preprocessing', CT), 
            ('estimator',RandomForestClassifier(n_estimators=100))
            ])

finish_pipe_RFR.fit(X, y)
print(finish_pipe_RFR.score(X,y))

In [ ]:
# Блок предсказания с использованием тестового набора

In [ ]:
y_predict = finish_pipe_RFR.predict(df1)
y_predict.shape

In [ ]:
# Маска отобраных номеров строк для предсказания из тестового набора
# не выбрасывал ничего, а предсказывал на таких данных, только немного преобразовав их
#mask
# Название вектора предсказанных значений  y_predict полученого на основании тестового набора
y_predict = y_predict